In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bs4 import BeautifulSoup    
import requests 
import re
import json

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def addItem(i, res, url): 
    response = requests.get(url)  

    # Теперь создадим объект BeautifulSoup, указывая html парсер    
    page = BeautifulSoup(response.text, 'html.parser')
    scripts = page.find_all('script')
    data = scripts[0].encode('latin1').decode('utf8')
    initial = page.find("script", {"id": "initial-state"})
    data2 = initial.encode('latin1').decode('utf8')
    these_regex=">(.+?)</"
    pattern=re.compile(these_regex)
    result = re.findall(pattern, data)
    jsonData = json.loads(result[0])
    result2 = re.findall(pattern, data2)
    jsonData2 = json.loads(result2[0])
    
    if 'mileage' not in jsonData2['card']['state']:
        jsonData2['card']['state']['mileage'] = 0
    if 'name' not in jsonData2['card']['vehicle_info']['complectation']:
        jsonData2['card']['vehicle_info']['complectation']['name'] = ''
    
    if 'price' not in jsonData['offers']:
        return res
    str1 = [jsonData['bodyType'],jsonData['brand'],jsonData['color'],url,jsonData['fuelType'],jsonData['modelDate'],jsonData['numberOfDoors'],jsonData['productionDate'],jsonData['vehicleTransmission'],
          jsonData['vehicleEngine']['engineDisplacement'],jsonData['vehicleEngine']['enginePower'],jsonData2['card']['state']['mileage'],
            jsonData2['card']['vehicle_info']['complectation']['name'],jsonData['offers']['price']]
    if i == 0:
        res = np.array(str1)
    else:
        res = np.vstack([res, str1])
    return res

In [ ]:
processed = list()
i = 0
res = list()
for carModel in ['volkswagen', 'nissan', 'bmw', 'mercedes', 'volvo', 'mitsubishi', 'skoda', 'toyota', 'audi']:
    for pageNum in range(1,3):
        url = 'https://auto.ru/cars/' + carModel + '/used/?page=' + str(pageNum)
        response = requests.get(url)
        page = BeautifulSoup(response.text, 'html.parser')
        scripts = page.find_all('a', href=re.compile(r"^https://auto.ru/cars/used/sale/"))
        for url in scripts:
            processed.append(url.get('href'))
            if processed.count(url.get('href')) == 1:
                res = addItem(i, res, url.get('href'))
                print('.', end='')
                i = i +1
        np.savetxt(
            './train_' + carModel + '.csv',
            res,
            fmt='%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s',
            header='bodyType,brand,color,url,fuelType,modelDate,numberOfDoors,productionDate,vehicleTransmission,engineDisplacement,enginePower,mileage,complectation_dict,price',
        )
        print(pageNum)